In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

ModuleNotFoundError: No module named 'plotly'

In [2]:
import json
groundtruth = "../ImageSets/groundtruth.json"

pt1 = open(groundtruth).read()
groundtruth = json.loads(pt1)

In [3]:
flight_ids = sorted(groundtruth["samples"].keys())
our_flights = flight_ids[0:394]

In [4]:
##### PART 1 #####
# Unnest JSON and store information in data frame
# initialize dictionary
flight_dict = {}

# initialize variables to store entity items
flight_id = 0
frame = -1
object_id = "No Object"
img_name = "No Image"
distance_m = -1

# initialize empty list
list1 = []

# loop through flights and save flight id as "key" and the nested structure as the "value"
for f_id in our_flights:
    flight = groundtruth["samples"][f_id]
    flight_dict[f_id] = flight
    
    # loop through each entity for each frame and save information to different variables
    for entity in flight["entities"]:
        
        flight_id = entity["flight_id"] if "flight_id" in entity else 0
        frame = entity["blob"]["frame"] if "frame" in entity["blob"] else -1
        object_id = entity["id"] if "id" in entity else "No Object"
        img_name = entity["img_name"] if "img_name" in entity else "No Image"
        distance_m = entity["blob"]["range_distance_m"] if "range_distance_m" in entity["blob"] else -1
        list1.append(["part1", flight_id, img_name, frame, object_id, distance_m])

# convert lists to a pandas data frame
cols = ["folder", "flight_id", "img_name", "frame", "object_id", "distance_m"]
df_pt1 = pd.DataFrame(list1, columns = cols)

In [5]:
file = df_pt1

In [8]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Airborne Object Appearances"),
    dcc.Graph(id = 'graph'),
    html.Label([
        "Please select the object you'd like to color:",
        dcc.Dropdown(
            id = "objects", clearable=False,
            value="Helicopter", options=[
                {"label":c, "value":c}
                for c in file["object_id"].unique()
            ])
    ]),
])
@app.callback(
    Output('graph', 'figure'),
    [Input("objects", "value")]
)
#file['color'] = 'red' if object_id == objects
def update_figure(drones):
    return px.bar(
        file,
        x = file["object_id"].value_counts(),
        y = file["object_id"].unique(),
        #color="color",
        #color_discrete_sequence=file.color.unique(),
        orientation = "h",
        labels = {"x": "Number of Frames", "y": "Airborne Object"},
        text_auto = ".2s",
        title="Number of Frame Appearances by Object"
    )
def update_layout(drones):
    return px.bar.update_layout(title = dict(x=0.5), paper_bgcolor="#3269a8")
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


In [10]:
??app.run_server